Practice using an autoencoder on an MNIST dataset
Dowload the MNIST dataset
Specify the neural network
Test reconstruction accuracy


In [1]:
# import the dataset: 

import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

from sklearn import datasets # Import the sklearn datasets module
from sklearn.datasets import fetch_openml # add this line to get th mnist dataset
import matplotlib.pyplot as plt




# In order to run this in class, we're going to reduce the dataset by a factor of 5
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)


# 70,000 records 

In [2]:
print("Full Dataset Size:", X.shape)

Full Dataset Size: (70000, 784)


In [4]:
# split the dataset into testing and training parts: 
X_train,X_test,y_tr,y_tst = train_test_split(X,y)
print("training", X_train.shape)
print("testing", X_test.shape)
type(y_tr[0])

training (52500, 784)
testing (17500, 784)


str

Download tensor package
convert dataset to tensor

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

X_train = torch.from_numpy(np.asarray(X_train))
X_test = torch.from_numpy(np.asarray(X_test))
y_train = y_tr.astype(np.float)
y_test=y_tst.astype(np.float)
y_train = torch.from_numpy(np.asarray(y_train)) 

y_test = torch.from_numpy(np.asarray(y_test))


X_train = X_train.to(torch.float32) # 
X_test = X_test.to(torch.float32)
y_train = y_train.to(torch.long)
y_test = y_test.to(torch.long)

In [6]:
# Describe the network structure **kwargs keeps the input general 

class AE(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.encoder_hidden_layer = nn.Linear(
            in_features=kwargs["input_shape"], out_features=128
        )
        self.encoder_output_layer = nn.Linear(
            in_features=128, out_features=128
        )
        self.decoder_hidden_layer = nn.Linear(
            in_features=128, out_features=128
        )
        self.decoder_output_layer = nn.Linear(
            in_features=128, out_features=kwargs["input_shape"]
        )

    def forward(self, features):
        activation = self.encoder_hidden_layer(features)
        activation = torch.relu(activation)
        code = self.encoder_output_layer(activation)
        code = torch.relu(code)                      # so, this is the latent space, consisting of 128 features? 
        activation = self.decoder_hidden_layer(code)
        activation = torch.relu(activation)
        activation = self.decoder_output_layer(activation)
        reconstructed = torch.relu(activation)
        return reconstructed
# so what is the "latent variable here? " - I think "code" represents the latent space/varaibles. 

In [8]:
#  use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# create a model from `AE` autoencoder class
# load it to the specified device, either gpu or cpu
model = AE(input_shape=784).to(device)

# create an optimizer object
# Adam optimizer with learning rate 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# mean-squared error loss
criterion = nn.MSELoss()

In [9]:
# load the data: 
# transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

# train_dataset = torchvision.datasets.MNIST(
#     root="~/torch_datasets", train=True, transform=transform, download=True)

# test_dataset = torchvision.datasets.MNIST(
#     root="~/torch_datasets", train=False, transform=transform, download=True)

# train_loader = torch.utils.data.DataLoader(
#     X_train, batch_size=128, shuffle=True, num_workers=4, pin_memory=True) # what does num_workers do? 

# test_loader = torch.utils.data.DataLoader(
#     test_dataset, batch_size=32, shuffle=False, num_workers=4)
from torch.utils.data import TensorDataset

training_data = TensorDataset(X_train,y_train)
test_data = TensorDataset(X_test,y_test)

batch_size = 256
train_loader = torch.utils.data.DataLoader(dataset=training_data,
                                           batch_size=batch_size, 
                                           shuffle=True)

batch_size = 256
test_loader = torch.utils.data.DataLoader(dataset=test_data,
                                           batch_size=batch_size, 
                                           shuffle=False)

In [10]:
epochs=40
loss=0
for epoch in range(epochs):
    loss = 0
    for batch_features, _ in train_loader:
        # reshape mini-batch data to [N, 784] matrix
        # load it to the active device
        batch_features = batch_features.view(-1, 784).to(device)
        
        # reset the gradients back to zero
        # PyTorch accumulates gradients on subsequent backward passes
        optimizer.zero_grad()
        
        # compute reconstructions
        outputs = model(batch_features)
        
        # compute training reconstruction loss
        train_loss = criterion(outputs, batch_features)
        
        # compute accumulated gradients
        train_loss.backward()
        
        # perform parameter update based on current gradients
        optimizer.step()
        
        # add the mini-batch training loss to epoch loss
        loss += train_loss.item()
    
    # compute the epoch training loss
    loss = loss / len(train_loader)
    
    # display the epoch training loss
    print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))

epoch : 1/40, loss = 2763.270143
epoch : 2/40, loss = 1327.143798
epoch : 3/40, loss = 1113.643034
epoch : 4/40, loss = 1011.562628
epoch : 5/40, loss = 950.037427
epoch : 6/40, loss = 908.574543
epoch : 7/40, loss = 875.041446
epoch : 8/40, loss = 850.229914
epoch : 9/40, loss = 831.515134
epoch : 10/40, loss = 814.384805
epoch : 11/40, loss = 801.481276
epoch : 12/40, loss = 789.629551
epoch : 13/40, loss = 779.425451
epoch : 14/40, loss = 769.297536
epoch : 15/40, loss = 761.237524
epoch : 16/40, loss = 753.788312
epoch : 17/40, loss = 745.497186
epoch : 18/40, loss = 739.219666
epoch : 19/40, loss = 731.695678
epoch : 20/40, loss = 727.542496
epoch : 21/40, loss = 720.688714
epoch : 22/40, loss = 715.613923
epoch : 23/40, loss = 711.234643
epoch : 24/40, loss = 705.824387
epoch : 25/40, loss = 701.862906
epoch : 26/40, loss = 697.881286
epoch : 27/40, loss = 694.419936
epoch : 28/40, loss = 691.345997
epoch : 29/40, loss = 686.548113
epoch : 30/40, loss = 685.919163
epoch : 31/40, 

In [12]:
# how do we get to the weights? and the outputs 
print(outputs.shape)


torch.Size([20, 784])
